# Contrôle des modèles décisionnels

Ce document a pour objectif de fournir des fichiers servant de base au contrôle de la complétude des modèles décisionnels (MD). 

Comme indiqué ci-dessous, plusieurs informations seront compilées à différentes échelles : 
- le nombre de cultures déclarées dans les leviers du MD
- le nombre de leviers total déclarés dans les rubriques du MD
- le nombre de catégories d'objectifs dans les rubriques du MD
- le nombre de levier par type de rubrique dans le MD

<img src="images/suggestion_diagramme.png" alt="drawing" width="600"/>

In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np

/home/bvuittenez/anaconda3/envs/agrosyst_entrepot/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
# ------------------ #
# IMPORT DES DONNÉES #
# ------------------ #
ENTREPOT_PATH = '~/Bureau/utils/data/'
df = {}

def import_df(df_name, path_data, sep, index_col=None):
    df[df_name] = pd.read_csv(path_data+df_name+'.csv', sep = sep, index_col=index_col, low_memory=False).replace({'\r\n': '\n'}, regex=True)

def import_dfs(df_names, path_data, sep = ',', index_col=None, verbose=False):
    for df_name in tqdm(df_names) : 
        if(verbose) :
            print(" - ", df_name)
        import_df(df_name, path_data, sep, index_col=index_col)

tables_with_id = [
    'modele_decisionnel', 
    'modele_decisionnel_maitrise', 
    'modele_decisionnel_strategie',
]

tables_without_id = [
    'modele_decisionnel_strategie_culture'
]

# import des données de l'entrepôt avec la colonne 'id' en index 
import_dfs(tables_with_id, ENTREPOT_PATH, sep = ',', index_col='id', verbose=False)

# import des données du magasin
import_dfs(tables_without_id, ENTREPOT_PATH, sep = ',', verbose=False)

### Constitution de modele_decisionnel_strategie_extanded
- ajout du nombre de cultures uniques

In [ ]:
# comptage du nombre de cultures différentes par stratégie :
df['modele_decisionnel_strategie_culture_count'] = df['modele_decisionnel_strategie_culture'].groupby('modele_decisionnel_strategie_id').agg({
    'culture_id' : pd.Series.nunique
}).rename(columns={'culture_id' : 'culture_count'}).fillna(0)

# NOTE : si on veut avoir la liste précise des culture utilisées pour pouvoir compter les "uniques" lors de futures agrégations, 
# on peut remplacer "pd.Series.nunique" par "pd.Series.unique"

In [ ]:
# faire remonter l'information jusqu'au modele_decisionnel_strategie
left = df['modele_decisionnel_strategie']
right = df['modele_decisionnel_strategie_culture_count']
df['modele_decisionnel_strategie_extanded'] = pd.merge(left, right, left_index=True, right_index=True, how='left')

### Constitution de modele_decisionnel_maitrise_extanded
- ajout du nombre de leviers uniques
- ajout du nombre de cultures totales

In [ ]:
# comptage du nombre de leviers différents par rubrique de maîtrises et somme du nombre de cultures
df['modele_decisionnel_maitrise_indicators_count'] = df['modele_decisionnel_strategie_extanded'].groupby('modele_decisionnel_maitrise_id').agg({
    'levier' : pd.Series.nunique,
    'culture_count' : 'sum' 
}).rename(columns={'levier' : 'levier_count'}).fillna(0)

# NOTE : si on veut avoir la liste précise des leviers pour pouvoir compter les "uniques" lors de futures agrégations, 
# on peut remplacer "pd.Series.nunique" par "pd.Series.unique"

In [ ]:
# faire remonter l'information jusqu'au modele_decisionnel_maitrise
left = df['modele_decisionnel_maitrise']
right = df['modele_decisionnel_maitrise_indicators_count']
df['modele_decisionnel_maitrise_extanded'] = pd.merge(left, right, left_index=True, right_index=True, how='left')

### Constitution de modele_decisionnel_extanded
- ajout du nombre de rubriques remplies
- ajout du nombre de leviers par rubriques
- ajout du nombre de leviers total
- ajout du nombre de catégories d'objectifs
- ajout du nombre de cultures totales

In [ ]:
# définition d'un dictionnaire de renommage pour plus de clarté par la suite
DICT_RENOMMAGE = {
    'ADVENTICES' : 'levier_ADVENTICE_count',
    'CYCLE_PLURIANNUEL_DE_CULTURE' : 'levier_CYCLE_PLURIANNUEL_DE_CULTURE_count',
    'FERTILITE_SOL_CULTURES' : 'levier_FERTILITE_SOL_CULTURES_count',
    'MAITRISE_DES_DOMMAGES_PHYSIQUES' : 'levier_MAITRISE_DES_DOMMAGES_PHYSIQUES_count',
    'MALADIES' : 'levier_MALADIES_count',
    'PRODUCTION' : 'levier_PRODUCTION_count',
    'RAVAGEURS' : 'levier_RAVAGEURS_count',
    'TRAVAIL_DU_SOL' : 'levier_TRAVAIL_DU_SOL_count'
}

In [ ]:
# comptage du nombre de leviers par type de rubrique pour chaque modèle décisionnel.
df['modele_decisionnel_leviers_count'] = df['modele_decisionnel_maitrise_extanded'].reset_index().groupby(['modele_decisionnel_id', 'type_rubrique']).agg({
    'levier_count' : 'sum'
}).rename(columns={'id' : 'rubrique_count', 'categorie_objectif' : 'categorie_objectif_count'}).fillna(0)

# pivot de la table pour obtenir une colonne par type de rubrique (voir le fonctionnement de pivot : https://pandas.pydata.org/docs/user_guide/reshaping.html)
df['modele_decisionnel_type_rubrique_count'] = df['modele_decisionnel_leviers_count'].reset_index().pivot(
    index='modele_decisionnel_id', 
    columns='type_rubrique',
    values='levier_count'
).fillna(0)

# comptage du nombre de rubriques pour chaque modèle décisionnel --> cette fois, on ne module plus par le type de rubrique.
df['modele_decisionnel_indicators_count'] = df['modele_decisionnel_maitrise_extanded'].reset_index().groupby(['modele_decisionnel_id']).agg({
    'id' : pd.Series.nunique,
    'levier_count' : 'sum',
    'categorie_objectif' : pd.Series.nunique
}).rename(columns={'id' : 'rubrique_count', 'categorie_objectif' : 'categorie_objectif_count'}).fillna(0)

# fusion des informations obtenues au sein d'un même dataframe avant ajout à la table principale 
left = df['modele_decisionnel_indicators_count']
right = df['modele_decisionnel_type_rubrique_count']
df['modele_decisionnel_indicators_count'] = pd.merge(left, right, left_index=True, right_index=True, how='left').rename(columns=DICT_RENOMMAGE)

In [ ]:
# faire remonter l'information jusqu'au modele_decisionnel
left = df['modele_decisionnel']
right = df['modele_decisionnel_indicators_count']
df['modele_decisionnel_extanded'] = pd.merge(left, right, left_index=True, right_index=True, how='left')

# on complète les indicateurs pour les lignes qui étaient absentes jusqu'à maintenant
indicators_to_fill = ['rubrique_count', 'levier_count', 'categorie_objectif_count'] + list(DICT_RENOMMAGE.values()) 

df['modele_decisionnel_extanded'][indicators_to_fill] =df['modele_decisionnel_extanded'][indicators_to_fill].fillna(0, inplace=False)

In [ ]:
# on obtient un lot de variable qui viennent directement qualifier nos modèles décisionnels !
df['modele_decisionnel_extanded']

### Constitution de sdc_extanded
- ajout de la variable have_OBSERVED
- ajout de la variable have_PLANNED
- ajout de la variable have_OBSERVED_or_PLANNED

TODO : ajout de toutes les informations précédentes --> simples agrégations comme vu précédemment

In [ ]:
# on qualifie les données plus précisément pour la suite
df['modele_decisionnel_extanded'].loc[:, 'is_OBSERVED'] = df['modele_decisionnel_extanded']['categorie'] == 'OBSERVED'
df['modele_decisionnel_extanded'].loc[:, 'is_PLANNED'] = df['modele_decisionnel_extanded']['categorie'] == 'PLANNED'

In [ ]:
# on obtient si il y a bien un modèle décisionnel de chaque type dans le sdc en groupant et en vérifiant qu'au moins un MD du type est True 
# NOTE : c'est ce que permet de faire "any", si on exigeait qu'ils soient tous à true, on mettrait "all"

df['modele_decisionnel_indicators'] = df['modele_decisionnel_extanded'].groupby('sdc_id').agg({
    'is_OBSERVED' : any,
    'is_PLANNED' : any
}).rename(columns={'is_OBSERVED' : 'have_OBSERVED', 'is_PLANNED' : 'have_PLANNED'})

df['modele_decisionnel_indicators'].loc[:, 'have_OBSERVED_or_PLANNED'] = df['modele_decisionnel_indicators']['have_OBSERVED'] | df['modele_decisionnel_indicators']['have_PLANNED']